In [ ]:
import numpy as np
import tensorflow as tf
import torch
import logging

In [ ]:
#Pytorch's version

def dynamic_k_matching(cost, pair_wise_ious, gt_classes, num_gt, fg_mask):
      # Dynamic K
      # ---------------------------------------------------------------
      matching_matrix = torch.zeros_like(cost, dtype=torch.uint8)
      print(f"matching matrix: {matching_matrix}")

      ious_in_boxes_matrix = pair_wise_ious
      n_candidate_k = min(10, ious_in_boxes_matrix.size(1))
      print(f"n_candidate_k: {n_candidate_k}, ious_in_boxes_matrix.size(1): {ious_in_boxes_matrix.size(1)}")
      topk_ious, _ = torch.topk(ious_in_boxes_matrix, n_candidate_k, dim=1)
      print(f"topk_ious: {topk_ious}")
      dynamic_ks = torch.clamp(topk_ious.sum(1).int(), min=1)
      dynamic_ks = dynamic_ks.tolist()
   
      print(f"dynamic_ks: {dynamic_ks}")

      for gt_idx in range(num_gt):
          _, pos_idx = torch.topk(
              cost[gt_idx], k=dynamic_ks[gt_idx], largest=False
          )
          print(f"pos_idx: {pos_idx}")
          matching_matrix[gt_idx][pos_idx] = 1

      print(f"matching_matrix: {matching_matrix}")

      del topk_ious, dynamic_ks, pos_idx

      anchor_matching_gt = matching_matrix.sum(0)
      print(f"anchor_matching_gt = {anchor_matching_gt}")

      # print(f"weird if statement: {(anchor_matching_gt > 1), (anchor_matching_gt > 1).sum()}")
      #(anchor_matching_gt > 1 creates a tensor of true/false e.g. [0, 1, 2, 3] -> [false, false, true ,true])
      #(anchor_matching_gt > 1).sum() returns a scalar of how many of the elements were true
      print(f"val for if statement: {(anchor_matching_gt > 1).sum()}")
      if (anchor_matching_gt > 1).sum() > 0:
          _, cost_argmin = torch.min(cost[:, anchor_matching_gt > 1], dim=0)
          print(f"cost_argmin: {cost_argmin}")
          print(f"anchor_matching_gt > 1: {anchor_matching_gt > 1}")

          matching_matrix[:, anchor_matching_gt > 1] *= 0
          print(f"matching_matrix: {matching_matrix}")

          matching_matrix[cost_argmin, anchor_matching_gt > 1] = 1
          print(f"matching_matrix: {matching_matrix}")

      # Continue from here
      fg_mask_inboxes = matching_matrix.sum(0) > 0

      num_fg = fg_mask_inboxes.sum().item()
      print(f"fg_mask_inboxes: {fg_mask_inboxes}, num_fg: {num_fg}\n")

      fg_mask[fg_mask.clone()] = fg_mask_inboxes
      print(f"fg_mask: {fg_mask}")
      
      matched_gt_inds = matching_matrix[:, fg_mask_inboxes].argmax(0)
      # print(f"matching_matrix[:, fg_mask_inboxes]: {matching_matrix[:, fg_mask_inboxes]}\n")
      print(f"matched_gt_inds: {matched_gt_inds}")

      gt_matched_classes = gt_classes[matched_gt_inds]
      print(f"gt_matched_classes: {gt_matched_classes}")
      
      pred_ious_this_matching = (matching_matrix * pair_wise_ious).sum(0)[
          fg_mask_inboxes
      ]
      print(f"pred_ious_this_matching: {pred_ious_this_matching}")
      return num_fg, gt_matched_classes, pred_ious_this_matching, matched_gt_inds

In [ ]:
# cost = torch.ones(8, 8)
cost = torch.tensor([[0.1081, 0.4863, 0.1679, 0.3832, 0.0141, 0.6604, 0.1300, 0.1762],
        [0.1400, 0.1980, 0.2087, 0.3757, 0.5950, 0.6953, 0.8711, 0.6065],
        [0.0214, 0.7250, 0.5053, 0.1129, 0.5049, 0.9175, 0.8503, 0.7133],
        [0.0845, 0.9322, 0.1333, 0.6464, 0.6768, 0.0621, 0.4914, 0.3330],
        [0.7293, 0.9651, 0.5887, 0.0382, 0.1010, 0.0868, 0.0012, 0.8953],
        [0.1871, 0.5527, 0.1132, 0.9443, 0.3355, 0.4003, 0.1088, 0.6429],
        [0.1602, 0.0666, 0.7899, 0.2332, 0.7720, 0.1733, 0.2885, 0.7663],
        [0.8238, 0.1027, 0.5462, 0.9049, 0.0366, 0.5497, 0.1775, 0.4196]])
pair_wise_ious = torch.ones(8, 8)
# pair_wise_ious = torch.tensor([[0.6325, 0.5803, 0.5652, 0.5558, 0.5212, 0.4099, 0.3997, 0.3588],
#         [0.7447, 0.6987, 0.5659, 0.4033, 0.3787, 0.2296, 0.1168, 0.0662],
#         [0.8883, 0.6677, 0.6483, 0.6067, 0.3392, 0.1518, 0.1076, 0.0939],
#         [0.9348, 0.7530, 0.6374, 0.5903, 0.3919, 0.2885, 0.2733, 0.0110],
#         [0.9011, 0.6985, 0.6685, 0.5251, 0.2050, 0.1975, 0.0348, 0.0318],
#         [0.9383, 0.9098, 0.8053, 0.7286, 0.7005, 0.2340, 0.1698, 0.0046],
#         [0.9737, 0.8708, 0.6161, 0.4645, 0.4458, 0.3038, 0.0854, 0.0497],
#         [0.7961, 0.5860, 0.5669, 0.5319, 0.4365, 0.3040, 0.1664, 0.1067]])
gt_classes = torch.tensor([2.0, 5.0, 63., 50., 9., 10., 70., 15.])
num_gt = 8
fg_mask = torch.ones(8, dtype=torch.bool)
dynamic_k_matching(cost, pair_wise_ious, gt_classes, num_gt, fg_mask)

matching matrix: tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]], dtype=torch.uint8)
n_candidate_k: 8, ious_in_boxes_matrix.size(1): 8
topk_ious: tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
dynamic_ks: [8, 8, 8, 8, 8, 8, 8, 8]
pos_idx: tensor([4, 0, 6, 2, 7, 3, 1, 5])
pos_idx: tensor([0, 1, 2, 3, 4, 7, 5, 6])
pos_idx: tensor([0, 3, 4, 2, 7, 1, 6, 5])
pos_idx: tensor([5, 0, 2, 7, 6, 3, 4, 1])
pos_idx: tensor([6, 3, 5, 4, 2, 0, 7, 1])
pos_idx: tensor([6, 2, 0, 4, 5, 1, 7, 3])
pos_

(8,
 tensor([63., 70., 10.,  9.,  2., 50.,  9.,  2.]),
 tensor([1., 1., 1., 1., 1., 1., 1., 1.]),
 tensor([2, 6, 5, 4, 0, 3, 4, 0]))

In [ ]:
anchor_matching_gt = torch.tensor([6, 3, 4, 2, 4, 3, 4, 0])
# matching_matrix = torch.tensor([[0, 0, 1, 0, 1, 0, 1, 0],
#         [1, 1, 1, 0, 0, 0, 0, 0],
#         [2, 0, 0, 1, 1, 0, 0, 0],
#         [3, 0, 1, 0, 0, 1, 0, 0],
#         [4, 0, 0, 1, 0, 1, 1, 0],
#         [5, 0, 1, 0, 1, 0, 1, 0],
#         [6, 1, 0, 0, 0, 1, 0, 0],
#         [7, 1, 0, 0, 1, 0, 1, 0]])
matching_matrix = []
for i in range(8):
  temp = []
  for j in range(8):
    temp.append(i*8 + j)
  matching_matrix.append(temp)
matching_matrix = torch.tensor(matching_matrix)
print(matching_matrix)

print(anchor_matching_gt > 1)
print(cost[:, anchor_matching_gt > 1])

cost_argmin = torch.tensor([2, 6, 5, 4, 0, 3, 4])
matching_matrix[cost_argmin, anchor_matching_gt > 1]
print(matching_matrix[cost_argmin, anchor_matching_gt > 1])

# print(matching_matrix)
# # matching_matrix[:, anchor_matching_gt > 1] *= 0
# print(matching_matrix[:, anchor_matching_gt > 1])
# print(matching_matrix)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7],
        [ 8,  9, 10, 11, 12, 13, 14, 15],
        [16, 17, 18, 19, 20, 21, 22, 23],
        [24, 25, 26, 27, 28, 29, 30, 31],
        [32, 33, 34, 35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44, 45, 46, 47],
        [48, 49, 50, 51, 52, 53, 54, 55],
        [56, 57, 58, 59, 60, 61, 62, 63]])
tensor([ True,  True,  True,  True,  True,  True,  True, False])
tensor([[0.1081, 0.4863, 0.1679, 0.3832, 0.0141, 0.6604, 0.1300],
        [0.1400, 0.1980, 0.2087, 0.3757, 0.5950, 0.6953, 0.8711],
        [0.0214, 0.7250, 0.5053, 0.1129, 0.5049, 0.9175, 0.8503],
        [0.0845, 0.9322, 0.1333, 0.6464, 0.6768, 0.0621, 0.4914],
        [0.7293, 0.9651, 0.5887, 0.0382, 0.1010, 0.0868, 0.0012],
        [0.1871, 0.5527, 0.1132, 0.9443, 0.3355, 0.4003, 0.1088],
        [0.1602, 0.0666, 0.7899, 0.2332, 0.7720, 0.1733, 0.2885],
        [0.8238, 0.1027, 0.5462, 0.9049, 0.0366, 0.5497, 0.1775]])
tensor([16, 49, 42, 35,  4, 29, 38])


In [ ]:
matching_matrix =[[1, 0, 1, 0, 1, 0, 1, 9],
                  [1, 0, 1, 0, 0, 0, 0, 9],
                  [1, 0, 0, 1, 1, 0, 0, 9],
                  [1, 0, 1, 0, 0, 1, 0, 9],
                  [0, 0, 0, 1, 0, 1, 1, 9],
                  [1, 0, 1, 0, 1, 0, 1, 9],
                  [1, 0, 0, 0, 0, 1, 0, 9],
                  [0, 0, 0, 0, 1, 0, 1, 9]]
matching_matrix = torch.tensor(matching_matrix)
fg_mask_inboxes = matching_matrix.sum(0) > 0
num_fg = fg_mask_inboxes.sum().item()
print(num_fg)
print(f"fg_mask_inboxes: {fg_mask_inboxes}, num_fg: {num_fg}\n")
fg_mask = tf.boolean_mask(fg_mask_inboxes, fg_mask)
## MAKE SURE ABOUT IT
print(f"fg_mask: {fg_mask}\n")

7
fg_mask_inboxes: tensor([ True, False,  True,  True,  True,  True,  True,  True]), num_fg: 7

fg_mask: [ True False  True  True  True  True  True  True]

